In [1]:
!pip install -qU \
  pinecone-client==2.2.1 \
  ipywidgets==7.0.0

!pip install torch
!pip install sentence_transformers

In [2]:
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM

from transformers import AutoTokenizer
import os
import nltk
from nltk.data import find

import transformers

import re
import torch

2023-11-20 00:34:38.779639: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 00:34:38.830826: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 00:34:38.830862: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 00:34:38.830897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 00:34:38.840437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 00:34:38.841187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [70]:
tokenizer = AutoTokenizer.from_pretrained('Himabindu/finetuned-t5-dialogstudio-npc')
model = AutoModelForSeq2SeqLM.from_pretrained('Himabindu/finetuned-t5-dialogstudio-npc')

In [5]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = encoder.encode(sentences)
print(embeddings)

[[ 6.76569045e-02  6.34959936e-02  4.87131327e-02  7.93049857e-02
   3.74480784e-02  2.65278551e-03  3.93749513e-02 -7.09847128e-03
   5.93614466e-02  3.15369666e-02  6.00980632e-02 -5.29051982e-02
   4.06067856e-02 -2.59308331e-02  2.98428163e-02  1.12689391e-03
   7.35148415e-02 -5.03817834e-02 -1.22386597e-01  2.37028226e-02
   2.97265183e-02  4.24768515e-02  2.56337449e-02  1.99515885e-03
  -5.69190681e-02 -2.71597914e-02 -3.29035334e-02  6.60248697e-02
   1.19007163e-01 -4.58791070e-02 -7.26214200e-02 -3.25840265e-02
   5.23413457e-02  4.50552963e-02  8.25298205e-03  3.67023982e-02
  -1.39415618e-02  6.53918907e-02 -2.64272150e-02  2.06396420e-04
  -1.36643639e-02 -3.62810567e-02 -1.95044018e-02 -2.89738141e-02
   3.94270308e-02 -8.84090886e-02  2.62424489e-03  1.36713963e-02
   4.83062752e-02 -3.11566219e-02 -1.17329173e-01 -5.11690639e-02
  -8.85288045e-02 -2.18962878e-02  1.42986439e-02  4.44167852e-02
  -1.34815518e-02  7.43392259e-02  2.66382582e-02 -1.98762771e-02
   1.79191

In [8]:


mini_lm_tokenizer = AutoTokenizer.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2"
)

# create token IDs for both sequences
token_ids = mini_lm_tokenizer(
    ["some text here", "some more text goes here too"],
    padding=True,
    return_attention_mask=False,
    return_token_type_ids=False
)
token_ids

{'input_ids': [[101, 2070, 3793, 2182, 102, 0, 0, 0], [101, 2070, 2062, 3793, 3632, 2182, 2205, 102]]}

In [10]:
# convert these token IDs into plain text tokens
for token_ids_list in token_ids["input_ids"]:
    print(mini_lm_tokenizer.convert_ids_to_tokens(token_ids_list))

['[CLS]', 'some', 'text', 'here', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', 'some', 'more', 'text', 'goes', 'here', 'too', '[SEP]']


In [48]:
from typing import List
import numpy as np

def embed_docs(docs: List[str]) -> List[List[float]]:
    out = encoder.encode(docs)
    return out.tolist()

In [14]:
!pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import pinecone

In [18]:
# add Pinecone API key from app.pinecone.io
api_key = "828c0ba7-fbe7-4f81-bd61-5b9c8ae0912a"
# set Pinecone environment - find next to API key in console
env = "gcp-starter"
pinecone.init(
    api_key=api_key,
    environment=env
)

In [19]:
import time

index_name = 'npc-rag'

if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
    
pinecone.create_index(
    name=index_name,
    dimension=embeddings.shape[1],
    metric='cosine'
)
# wait for index to finish initialization
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

In [24]:
# Load the dataset
npc_train = load_dataset("amaydle/npc-dialogue", split="train")
npc_test = load_dataset("amaydle/npc-dialogue", split="test")

# Automatically splits it into train and test for you - let's ignore that for now and just combine them as one

# First, transform them into pandas DFs
train = pd.DataFrame(data = {'name': npc_train['Name'], 'bio':npc_train['Biography'], 'query':npc_train['Query'], 'response':npc_train['Response'], 'emotion':npc_train['Emotion']})
test = pd.DataFrame(data = {'name': npc_test['Name'], 'bio':npc_test['Biography'], 'query':npc_test['Query'], 'response':npc_test['Response'], 'emotion':npc_test['Emotion']})

# Now combine into a single df
npc = pd.concat([train, test])
npc

# Create a character-level dataset, since the characters show up multiple times in the dataset
character_level_dataset = npc[['name', 'bio']]
character_level_dataset.drop_duplicates(inplace=True)
character_level_dataset['answers'] = character_level_dataset['name'] + ' ' + character_level_dataset['bio']
character_level_dataset = character_level_dataset[['answers']]
character_level_dataset = pd.concat([character_level_dataset, pd.DataFrame(data={'answers':['The Magic Key is under the bridge', 'The sword is in the cave']})])

Found cached dataset parquet (/home/ec2-user/.cache/huggingface/datasets/amaydle___parquet/amaydle--npc-dialogue-90a44e1b412a95be/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/ec2-user/.cache/huggingface/datasets/amaydle___parquet/amaydle--npc-dialogue-90a44e1b412a95be/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
/tmp/ipykernel_4116/3469310117.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  character_level_dataset.drop_duplicates(inplace=True)
/tmp/ipykernel_4116/3469310117.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [49]:
from tqdm.auto import tqdm

batch_size = 2  # can increase but needs larger instance size otherwise instance runs out of memory
vector_limit = 1000

answers = character_level_dataset
index = pinecone.Index(index_name)

for i in tqdm(range(0, len(answers), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(answers))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in answers["answers"][i:i_end]]
    # create embeddings
    texts = answers["answers"][i:i_end].tolist()
    embeddings = embed_docs(texts)
    # create records list for upsert
    records = zip(ids, embeddings, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [00:05<00:00, 10.06it/s]


In [25]:
character_level_dataset

,answers
0,Bikram Bikram is a rough and tough smuggler fr...
1,Arcturus the Bounty Hunter Arcturus is a fearl...
2,Elena Gilbert Elena Gilbert is a teenage girl ...
3,Arin the Cleric Arin is a devoted follower of ...
4,Scott McCall Scott McCall is a teenage werewol...
...,...
634,"Tony Stark Tony Stark, genius inventor and bil..."
646,Thorne Thorne is a mysterious elf who has live...
1493,Lara Croft Lara Croft is a British archaeologi...
0,The Magic Key is under the bridge


In [50]:
# check number of records in the index
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00104,
 'namespaces': {'': {'vector_count': 104}},
 'total_vector_count': 104}

In [93]:
question = ["What powers does you have?"]

In [94]:
# extract embeddings for the questions
query_vec = embed_docs(question)[0]

# query pinecone
res = index.query(query_vec, top_k=5, include_metadata=True)

# show the results
res

{'matches': [{'id': '16',
              'metadata': {'text': 'Elsa Elsa is the Snow Queen of Arendelle, '
                                   'with the power to control ice and snow. '
                                   'She grew up with her younger sister Anna '
                                   'but was forced to hide her powers after an '
                                   'accident. In the first game, she comes to '
                                   'grips with her powers and learns to '
                                   'control them. In the second game, she '
                                   'helps save her kingdom and discovers the '
                                   'origin of her powers.'},
              'score': 0.780111194,
              'values': []},
             {'id': '21',
              'metadata': {'text': 'Kaela the Enchantress Kaela is a powerful '
                                   'enchantress who was born with a natural '
                                   'a

In [97]:
max_section_len = 1000
separator = "\n"

def construct_context(contexts: List[str]) -> str:
    chosen_sections = []
    chosen_sections_len = 1
    
    chosen_sections = contexts[:chosen_sections_len]
    concatenated_doc = separator.join(chosen_sections)
    print(
        f"With maximum sequence length {max_section_len}, selected top {len(chosen_sections)} document sections: \n{concatenated_doc}"
    )
    return concatenated_doc

In [98]:

context_str = construct_context(contexts=contexts)

With maximum sequence length 1000, selected top 1 document sections: 
Elsa Elsa is the Snow Queen of Arendelle, with the power to control ice and snow. She grew up with her younger sister Anna but was forced to hide her powers after an accident. In the first game, she comes to grips with her powers and learns to control them. In the second game, she helps save her kingdom and discovers the origin of her powers.


In [99]:
text_input = context_str + question[0]
test_tokenized = tokenizer.encode_plus(text_input, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

out = model.generate(
  input_ids=test_input_ids,attention_mask=test_attention_mask
)

tokenizer.decode(out[0])

'<pad> Elsa has the power to control the elements and helps her navigate through difficult situations. '

{'input_ids': [1289, 7, 9, 1289, 7, 9, 19, 8, 9394, 5286, 13, 1521, 727, 693, 6, 28, 8, 579, 12, 610, 3, 867, 11, 2983, 5, 451, 3, 4774, 95, 28, 160, 5868, 4806, 7588, 68, 47, 5241, 12, 7387, 160, 11552, 227, 46, 3125, 5, 86, 8, 166, 467, 6, 255, 639, 12, 7411, 7, 28, 160, 11552, 11, 669, 7, 12, 610, 135, 5, 86, 8, 511, 467, 6, 255, 1691, 1097, 160, 14740, 11, 2928, 7, 8, 5233, 13, 160, 11552, 5, 5791, 5434, 5634, 4055, 49, 5791, 5434, 19, 3, 9, 985, 18, 10386, 620, 52, 113, 3, 4774, 95, 16, 8, 5827, 28, 160, 3, 15, 40, 1926, 2039, 11, 936, 2353, 5, 451, 65, 3, 9, 19894, 333, 21, 1405, 11, 3, 9, 1659, 2135, 12, 8, 3127, 13, 8, 5827, 5, 5791, 5434, 19, 1704, 18, 7820, 1054, 11, 2547, 6, 68, 54, 36, 26246, 17, 1329, 13, 1067, 52, 7, 788, 12, 160, 95, 3770, 5, 1699, 15, 521, 1699, 15, 521, 19, 3, 9, 1817, 9781, 3249, 3, 10386, 113, 3, 4774, 95, 16, 8, 3, 35, 8694, 1054, 15099, 13, 1289, 1926, 5012, 5, 451, 65, 3, 9, 4816, 3, 7820, 11, 3, 9, 22071, 6794, 6, 68, 54, 36, 882, 123, 9416, 116,